# 02_ingest_and_query.ipynb — Ingest & Query Check
Notebook này giúp bạn **kiểm tra toàn bộ pipeline Ngày 2**:  
1) Trích xuất & tiền xử lý văn bản, 2) Chunk **xuyên trang**, 3) Ingest vào **Chroma**, 4) **Query** (dense) và (tuỳ chọn) **hybrid**.

> ⚠️ Yêu cầu: bạn đã cài thư viện, điền `.env`, và đặt PDF vào `data/raw/`.


## 0. Cấu hình đường dẫn & tham số
- `PDF_PATH`: đường dẫn tới PDF.
- `COLLECTION_NAME`: tên collection Chroma để lưu/đọc.
- `MAX_TOKENS`, `OVERLAP`: tham số chunk xuyên trang. Khớp với lệnh bạn đã dùng khi ingest (VD: `--max_tokens 300 --overlap 100`).


In [ ]:
PDF_PATH = 'data/raw/PrisonersOfGeography-TimMarshall.pdf'  # đổi nếu cần
COLLECTION_NAME = 'prisoners_of_geography'       # đổi nếu bạn dùng tên khác

# Tham số chunk (nếu bạn muốn xem pipeline trước khi add vào Chroma)
MAX_TOKENS = 300
OVERLAP = 100

## 1. Nạp `.env` & import các hàm tiện ích
- Nạp biến môi trường (đường dẫn Chroma, model, v.v.).
- Import hàm đọc PDF và các hàm tiền xử lý/chunk.


In [ ]:
from dotenv import load_dotenv
_ = load_dotenv()

import os, itertools
from src.utils.pdf_loader import extract_text_by_page
from src.utils.text_preprocess import clean_text, split_paragraphs_v2, chunk_across_pages
from src.utils.chroma_client import get_chroma_collection

# Kiểm tra PDF tồn tại
assert os.path.exists(PDF_PATH), f'Không tìm thấy PDF tại: {PDF_PATH}'

## 2. Trích xuất văn bản theo trang (preview)
- Gọi `extract_text_by_page` để thu về danh sách `{page, text}`.
- Xem nhanh **số trang** và nội dung **một vài trang mẫu** để đảm bảo dấu/line-break ổn.


In [ ]:
pages = extract_text_by_page(PDF_PATH)
print('Tổng số trang:', len(pages))

SAMPLE_PAGES = [1, 25, 100]  # thay đổi nếu cần
for p in pages:
    if p['page'] in SAMPLE_PAGES:
        print('='*80)
        print(f"Trang {p['page']}")
        print('-'*80)
        print((p['text'][:1000] + ' ...') if len(p['text']) > 1000 else p['text'])

## 3. Tiền xử lý & tách đoạn, sau đó **chunk xuyên trang**
- `clean_text`: gom khoảng trắng, rút gọn xuống dòng.
- `split_paragraphs_v2`: tách **đoạn**; nếu không có `\n\n` thì fallback theo **câu**.
- `chunk_across_pages`: gom **toàn bộ đoạn/câu từ nhiều trang** tới khi đạt `MAX_TOKENS`, có `OVERLAP` (chồng lặp) giữa các chunk liên tiếp.


In [ ]:
# Gom toàn bộ (page, paragraph)
page_paragraphs = []
for p in pages:
    text = clean_text(p['text'])
    paras = split_paragraphs_v2(text)
    for para in paras:
        page_paragraphs.append((p['page'], para))

# Tạo chunk xuyên trang
book_chunks = chunk_across_pages(page_paragraphs, max_tokens=MAX_TOKENS, overlap=OVERLAP)
len(book_chunks)

### 3.1. Xem vài chunk đầu để xác nhận `start_page/end_page` & độ dài
- Kiểm tra xem chunk có **bắt đầu/kết thúc** đúng ranh giới trang mong đợi không.
- Kiểm tra **số từ** để đảm bảo cỡ chunk đúng `MAX_TOKENS` (xấp xỉ).


In [ ]:
def wc(s): return len((s or '').split())

for i, ch in enumerate(book_chunks[:5], 1):
    print(f"#{i} pages {ch['start_page']}→{ch['end_page']} | words: {wc(ch['text'])}")
    print((ch['text'][:400] + ' ...') if len(ch['text']) > 400 else ch['text'])
    print('-'*100)

## 4. Ingest vào Chroma (nếu bạn **chưa** ingest bằng script)
> Nếu bạn **đã ingest bằng lệnh CLI** trước đó, bạn có thể **bỏ qua cell này**.

- Tạo collection và add `documents + metadatas + ids`.
- Metadata dùng `start_page/end_page` (và `chapter` nếu bạn có `eval/chapter_map.json`).


In [ ]:
# (TÙY CHỌN) Ingest nhanh ngay trong notebook
# Cảnh báo: nếu collection đã có dữ liệu, cần cẩn thận để tránh add trùng.
DO_INGEST_HERE = False

if DO_INGEST_HERE:
    import json
    # Tải chapter map nếu có
    cmap_path = 'eval/chapter_map.json'
    cmap = {}
    if os.path.exists(cmap_path):
        with open(cmap_path, 'r', encoding='utf-8') as f:
            cmap = json.load(f)

    def resolve_chapter_for_span(start_page: int, end_page: int, chapter_map):
        for rng, name in chapter_map.items():
            a, b = rng.split('-')
            try: a, b = int(a), int(b)
            except ValueError: continue
            if a <= start_page <= b:
                return name
        return ''

    col = get_chroma_collection(COLLECTION_NAME)
    all_docs, metadatas, ids = [], [], []
    for idx, ch in enumerate(book_chunks):
        all_docs.append(ch['text'])
        chap = resolve_chapter_for_span(ch['start_page'], ch['end_page'], cmap) if cmap else ''
        metadatas.append({
            'start_page': ch['start_page'],
            'end_page': ch['end_page'],
            'chapter': chap,
            'source': os.path.basename(PDF_PATH),
        })
        ids.append(f"{ch['start_page']}-{ch['end_page']}-{idx}")

    if all_docs:
        col.add(documents=all_docs, metadatas=metadatas, ids=ids)
        print(f"Ingested {len(all_docs)} chunks into collection '{COLLECTION_NAME}'.")

## 5. Kiểm tra collection: đếm số chunk & xem mẫu metadata
- Kết nối collection, **đếm số chunk**.
- Lấy một vài **document + metadata** để xem nhanh.


In [ ]:
col = get_chroma_collection(COLLECTION_NAME)

try:
    total = col.count()
except Exception:
    total = len(col.get()['ids'])
print('Tổng số chunk trong collection:', total)

sample = col.get(limit=5)
for doc, meta, _id in zip(sample['documents'], sample['metadatas'], sample['ids']):
    print('ID:', _id)
    print('Pages:', meta.get('start_page'), '→', meta.get('end_page'))
    print('Chapter:', meta.get('chapter'))
    print('Source:', meta.get('source'))
    print('Text preview:', (doc[:400] + ' ...') if len(doc) > 400 else doc)
    print('='*100)

## 6. Thống kê độ dài chunk — kiểm tra cỡ chunk thực tế
- Cho biết **min / max / trung bình số từ** mỗi chunk.
- Xem **Top-5 chunk dài nhất**.


In [ ]:
got = col.get()
docs = got['documents']
metas = got['metadatas']

lengths = [len((d or '').split()) for d in docs]
print('Số chunk:', len(lengths))
print('Min/Max/Trung bình:', min(lengths), max(lengths), round(sum(lengths)/len(lengths), 1))

# Top-5 chunk dài nhất
longest_idx = sorted(range(len(lengths)), key=lambda i: lengths[i], reverse=True)[:5]
for i in longest_idx:
    m = metas[i]
    print(f"idx={i} | words={lengths[i]} | pages {m.get('start_page')}→{m.get('end_page')} | chapter={m.get('chapter')}")

## 7. Sắp xếp theo trang & kiểm tra **overlap** (tham khảo)
- Sắp xếp chunk theo `(start_page, end_page)` để đảm bảo **trật tự hợp lý**.
- Ước lượng mức **overlap** thô giữa 2 chunk liên tiếp (so sánh 100 từ cuối/đầu).


In [ ]:
rows = []
for d, m in zip(docs, metas):
    rows.append({
        'text': d,
        'start': m.get('start_page', 0),
        'end': m.get('end_page', 0),
        'chapter': m.get('chapter', '')
    })
rows = sorted(rows, key=lambda r: (r['start'], r['end']))

def head_words(s, n=100): 
    w = (s or '').split()
    return w[:n]

def tail_words(s, n=100): 
    w = (s or '').split()
    return w[-n:]

def overlap_ratio(a_tail, b_head):
    if not a_tail or not b_head: return 0.0
    set_a, set_b = set(a_tail), set(b_head)
    inter = len(set_a & set_b)
    return inter / max(1, len(set_a))

for k in range(min(10, len(rows)-1)):
    a, b = rows[k], rows[k+1]
    r = overlap_ratio(tail_words(a['text'], 100), head_words(b['text'], 100))
    print(f"Pair {k}: pages ({a['start']}→{a['end']}) -> ({b['start']}→{b['end']}) | overlap≈{r:.2f}")

## 8. Thử **query dense** trực tiếp với Chroma
- Gọi `collection.query(query_texts=[q], n_results=k)` để xem **top-k context** có đúng chương/trang.
- Đây **chưa phải** hybrid; chỉ là kiểm tra nhanh chất lượng embeddings.


In [ ]:
q = "Ví dụ: Hỏi một câu về địa chính trị Nga"  # thay bằng câu của bạn
k = 5
res = col.query(query_texts=[q], n_results=k)
for i, (doc, meta) in enumerate(zip(res['documents'][0], res['metadatas'][0]), 1):
    print(f"#{i} pages {meta.get('start_page')}→{meta.get('end_page')} | {meta.get('chapter')}")
    print((doc[:300] + ' ...') if len(doc) > 300 else doc)
    print('-'*80)

## 9. (Tuỳ chọn) Thử **hybrid retrieve** nếu đã có `rag_pipeline.hybrid_retrieve`
- Hybrid = **BM25 (sparse)** + **Dense (Chroma)**, thường cho độ phủ tốt hơn với **từ khoá địa danh**.
- Nếu import lỗi, nghĩa là file `rag_pipeline.py` của bạn chưa có hàm này (bỏ qua cell).


In [ ]:
try:
    from src.rag_pipeline import hybrid_retrieve
    q = "Ví dụ: Vì sao vị trí địa lý ảnh hưởng đến chiến lược của Nga?"
    hits = hybrid_retrieve(q, k_dense=5, k_sparse=5, collection_name=COLLECTION_NAME)
    for i, h in enumerate(hits, 1):
        meta = h.get('meta', {})
        print(f"#{i} src={h.get('src')} norm={round(h.get('norm', 3), 3)} | pages {meta.get('start_page')}→{meta.get('end_page')} | {meta.get('chapter')}")
        print((h['text'][:300] + ' ...') if len(h['text']) > 300 else h['text'])
        print('-'*80)
except Exception as e:
    print('Không thể chạy hybrid_retrieve (bỏ qua nếu bạn chưa triển khai):', e)

---
### Ghi chú cuối
- Nếu kết quả **không đúng chương/trang** như mong muốn:
  - Thử `MAX_TOKENS` {250–450} và `OVERLAP` {80–120}.
  - Kiểm tra `eval/chapter_map.json` có khớp bản PDF của bạn không.
  - Với câu “tên riêng/địa danh”, **hybrid** thường giúp kết quả ổn định hơn.
